In [2]:
!pwd
!ls

/d/GH/GitWorkSpace/bank_model_competiton/data/v21
bak
model_gbdt_v21.ipynb
model_lgb_v21.ipynb
process_v21.ipynb


In [62]:
import pandas as pd 
import matplotlib.pyplot as plt
import statistics
import datetime
import seaborn as sns
import os
import numpy as np
import time 
from sklearn.preprocessing import OrdinalEncoder

pd.set_option('display.max_columns',100)

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

suffix = os.path.split(os.getcwd())[-1]

root_dir='../../'

train_path=root_dir+'train.csv'
train_tx_path=root_dir+'train_bank_statement.csv'
test_path=root_dir+'testaa.csv'
test_tx_path=root_dir+'testaa_bank_statement.csv'

print(suffix)
print('process time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))

v21
process time :  2025-08-31 16:22:52


In [63]:
mode="train"

NROWS = 10000000000000000

if mode == "train":
    base_input_path = train_path
    tx_input_path = train_tx_path
    output_path = "train.dat.%s" % suffix
else:
    base_input_path = test_path
    tx_input_path = test_tx_path
    output_path = "test.dat.%s" % suffix

print(mode, base_input_path, tx_input_path, output_path)

train ../../train.csv ../../train_bank_statement.csv train.dat.v21


#  基础特征

In [72]:
!pwd
df = pd.read_csv(base_input_path, index_col=['id'], nrows=NROWS)
# print(df.head())
print(df.shape)

df['balance_account_avg'] = df['balance'] / df['balance_accounts']
df['loan_term_avg'] = df['loan'] / df['term']
df['balance_accounts_ratio'] = df['balance_accounts'] / df['total_accounts']

#log 处理
df['loan_log'] = df['loan'].apply(lambda x : np.log(x))
df['balance_account_avg_log'] = df['balance_account_avg'].apply(lambda x : np.log(x))
df['loan_term_avg_log'] = df['loan_term_avg'].apply(lambda x : np.log(x))
df['balance_accounts_ratio_log'] = df['balance_accounts_ratio'].apply(lambda x : np.log(x))
df['interest_rate_log'] = df['interest_rate'].apply(lambda x : np.log(x))

df['balance_log'] = df['balance'].apply(lambda x : np.log(x))
df['balance_limit_log'] = df['balance_limit'].apply(lambda x : np.log(x))
df['balance_accounts_log'] = df['balance_accounts'].apply(lambda x : np.log(x))

# zip code
df['zip_province']  = df['zip_code'].apply(lambda x : str(x)[:2])
df['zip_city']      = df['zip_code'].apply(lambda x : str(x)[:4])

#level 处理
df['level_hash'] = df['level'].apply(lambda x : hash(x) % 1000 )
encoder = OrdinalEncoder(categories=[['A0','A1','A2','A3','A4','A5','B0','B1','B2','B3','B4','B5','C0','C1','C2','C3','C4','C5','D0','D1','D2','D3','D4','D5','E0','E1','E2','E3','E4','E5']]) 
df['level_ord'] = encoder.fit_transform(df[['level']].values)

df['grade'] = df['level'].apply( lambda x : str(x)[0])  #提取ABCDE
df['grade'] = df['grade'].map({'A':1 , 'B':2, 'C':3, 'D':4, 'E':5})


# 分桶处理
bins_num = 50
tmp_labels = ['%d' % i for i in range(bins_num)]
df['interest_rate_cut'] = pd.cut(df['interest_rate'], bins=bins_num, labels = tmp_labels)

tmp_labels = ['%d' % i for i in range(bins_num)]
df['interest_rate_log_cut'] = pd.cut(df['interest_rate_log'], bins=bins_num, labels = tmp_labels)

# 等频
tmp_labels = ['%d' % i for i in range(bins_num)]
df['balance_cut'] = pd.qcut(df['balance'], q=bins_num, labels = tmp_labels, duplicates='drop')

df['loan_cut'] = pd.qcut(df['loan'], q=bins_num,  duplicates='drop')
loan_labels = np.unique(df['loan_cut'].values.codes).tolist()
df['loan_cut'] = pd.qcut(df['loan'], q=bins_num,  labels = loan_labels,  duplicates='drop')

tmp_labels = ['%d' % i for i in range(bins_num)]
df['balance_limit_cut'] = pd.qcut(df['balance_limit'], q=bins_num, labels = tmp_labels, duplicates='drop')

df['loan_term_avg_cut'] = pd.qcut(df['loan_term_avg'], q=bins_num,  duplicates='drop')
loan_labels = np.unique(df['loan_term_avg_cut'].values.codes).tolist()
df['loan_term_avg_cut'] = pd.qcut(df['loan_term_avg'], q=bins_num,  labels = loan_labels,  duplicates='drop')

tmp_labels = ['%d' % i for i in range(bins_num)]
df['balance_account_avg_cut'] = pd.qcut(df['balance_account_avg'], q=bins_num, labels = tmp_labels, duplicates='drop')

# 时间处理
df['record_time_format'] = df['record_time'].apply(lambda x: datetime.datetime.fromtimestamp(x))
df['record_time_year'] = df['record_time_format'].map(lambda x : x.year)
df['record_time_month'] = df['record_time_format'].map(lambda x : x.month)      # 探索周期性
df['record_time_week'] = df['record_time_format'].map(lambda x : x.week)      # 探索周期性
df['record_time_year_month'] = df['record_time_format'].map(lambda x : x.strftime('%Y%m'))

df.drop(['record_time_format'], axis=1, inplace=True)
df

/d/GH/GitWorkSpace/bank_model_competiton/data/v21
(53480, 18)


,title,career,zip_code,residence,loan,term,interest_rate,issue_time,syndicated,installment,record_time,history_time,total_accounts,balance_accounts,balance_limit,balance,level,label,balance_account_avg,loan_term_avg,balance_accounts_ratio,loan_log,balance_account_avg_log,loan_term_avg_log,balance_accounts_ratio_log,interest_rate_log,balance_log,balance_limit_log,balance_accounts_log,zip_province,zip_city,level_hash,level_ord,grade,interest_rate_cut,interest_rate_log_cut,balance_cut,loan_cut,balance_limit_cut,loan_term_avg_cut,balance_account_avg_cut,record_time_year,record_time_month,record_time_week,record_time_year_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,9,0.0,221373,1,7200,36,10.95,1238631967,0,1,1238630622,472006661,17.0,9.0,36200.0,13856.00,A4,0,1539.555556,200.000000,0.529412,8.881836,7.339249,5.298317,-0.635989,2.393339,9.536474,10.496814,2.197225,22,2213,625,4.0,1,10,20,40,4,43,6,42,2009,4,14,200904
1,8,10.0,311681,0,21300,36,12.95,1128212052,0,0,1161907665,763779041,17.0,9.0,20400.0,13773.00,B0,1,1530.333333,591.666667,0.529412,9.966462,7.333241,6.382943,-0.635989,2.561096,9.530465,9.923290,2.197225,31,3116,54,6.0,2,14,25,40,19,35,20,42,2006,10,43,200610
2,8,7.0,271562,1,10400,60,21.05,1249171509,0,0,1383958593,727143443,17.0,9.0,10800.0,2023.00,B4,0,224.777778,173.333333,0.529412,9.249561,5.415112,5.155217,-0.635989,3.046901,7.612337,9.287301,2.197225,27,2715,403,10.0,2,30,38,19,10,29,4,1,2013,11,45,201311
3,7,2.0,522083,0,33050,36,16.40,1172882234,0,1,1214353935,687660346,17.0,9.0,24700.0,21992.00,B3,0,2443.555556,918.055556,0.529412,10.405777,7.801209,6.822258,-0.635989,2.797281,9.998434,10.114559,2.197225,52,5220,616,9.0,2,21,31,45,22,38,27,46,2008,6,26,200806
4,8,3.0,101026,1,5200,36,14.35,1172882384,0,0,1240274527,322012875,17.0,9.0,5100.0,1669.00,B2,1,185.444444,144.444444,0.529412,8.556414,5.222755,4.972895,-0.635989,2.663750,7.419980,8.536996,2.197225,10,1010,222,8.0,2,17,28,16,1,24,1,1,2009,4,17,200904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53475,2,2.0,603000,1,9000,12,23.55,1172880000,0,0,1157587200,1061769600,12.0,5.0,3535.0,2595.73,A4,0,519.146000,750.000000,0.416667,9.104980,6.252185,6.620073,-0.875469,3.159126,7.861623,8.170469,1.609438,60,6030,625,4.0,1,35,42,23,7,20,23,9,2006,9,36,200609
53476,0,10.0,601702,1,8000,12,30.70,1160092800,0,0,1138665600,1038268800,5.0,2.0,1965.0,1433.34,B2,0,716.670000,666.666667,0.400000,8.987197,6.574615,6.502290,-0.916291,3.424263,7.267763,7.583248,0.693147,60,6017,222,8.0,2,48,49,13,5,12,21,18,2006,1,5,200601
53477,2,10.0,602808,1,10000,12,9.40,1180310400,0,0,1108771200,1087603200,12.0,5.0,7253.0,3813.79,B2,0,762.758000,833.333333,0.416667,9.210340,6.636941,6.725434,-0.875469,2.240710,8.246379,8.889170,1.609438,60,6028,222,8.0,2,7,16,27,9,26,25,21,2005,2,7,200502


#  交易特征处理

In [73]:
df_tx = pd.read_csv(tx_input_path, nrows = NROWS, index_col=['id'])
df_tx['time_format'] = df_tx['time'].apply(lambda x: datetime.datetime.fromtimestamp(x))
print(df_tx)

             time  direction       amount         time_format
id                                                           
0      1224115200          0  8771.350000 2008-10-16 08:00:00
0      1224288000          1   310.650000 2008-10-18 08:00:00
0      1224460800          1   152.620000 2008-10-20 08:00:00
0      1225152000          1    20.490000 2008-10-28 08:00:00
0      1226793600          1   173.170000 2008-11-16 08:00:00
...           ...        ...          ...                 ...
50371  1159747200          1   133.818723 2006-10-02 08:00:00
50371  1160179200          1    71.777279 2006-10-07 08:00:00
50371  1160956800          0    46.981217 2006-10-16 08:00:00
50371  1161129600          1   160.510592 2006-10-18 08:00:00
50371  1161475200          1    89.648090 2006-10-22 08:00:00

[1717418 rows x 4 columns]


In [74]:
#days_diff, tm_count, total_amount, amount_1, amount_0, total_amount_day_avg, amount_1_day_avg, amount_0_day_avg
print(df_tx.shape)

df_tx_stat = pd.DataFrame()
df_tx_stat['tx_time_max'] = df_tx.groupby('id')['time_format'].agg('max')
df_tx_stat['tx_time_min'] = df_tx.groupby('id')['time_format'].agg('min')

df_tx_stat['tx_max_min_days'] = (df_tx_stat['tx_time_max'] - df_tx_stat['tx_time_min'])
df_tx_stat['tx_max_min_days'] = df_tx_stat['tx_max_min_days'].map(lambda x : x.days)

df_tx_stat['tx_count'] = df_tx.groupby('id')['amount'].agg('count')
df_tx_stat['total_amount'] = df_tx.groupby('id')['amount'].agg('sum')

df_tx_stat['1_amount'] = df_tx.groupby('id').apply( lambda x : x [ x['direction'] == 1]['amount'].sum())
df_tx_stat['0_amount'] = df_tx.groupby('id').apply( lambda x : x [ x['direction'] == 0]['amount'].sum())

df_tx_stat['total_amount_avg']  = df_tx_stat['total_amount'] / df_tx_stat['tx_max_min_days'] 
df_tx_stat['1_amount_avg']  = df_tx_stat['1_amount'] / df_tx_stat['tx_max_min_days'] 
df_tx_stat['0_amount_avg']  = df_tx_stat['0_amount'] / df_tx_stat['tx_max_min_days'] 

df_tx_stat['total_amount_avg2']  = df_tx_stat['total_amount'] / df_tx_stat['tx_count'] 
df_tx_stat['1_amount_avg2']  = df_tx_stat['1_amount'] / df_tx_stat['tx_count'] 
df_tx_stat['0_amount_avg2']  = df_tx_stat['0_amount'] / df_tx_stat['tx_count'] 

#交易活跃度： 效果次数/天数
df_tx_stat['tx_count_avg']  = df_tx_stat['tx_count'] / df_tx_stat['tx_max_min_days'] 
df_tx_stat['tx_tmstp_max'] = df_tx.groupby('id')['time'].agg('max')
df_tx_stat['tx_tmstp_min'] = df_tx.groupby('id')['time'].agg('min')
df_tx_stat['tx_max_min_days'] = (df_tx_stat['tx_time_max'] - df_tx_stat['tx_time_min'])
df_tx_stat['tx_max_min_days'] = df_tx_stat['tx_max_min_days'].map(lambda x : x.days)

df_tx_stat.drop([ 'tx_time_max', 'tx_time_min'], axis = 1, inplace=True)
df_tx_stat

(1717418, 4)


,tx_max_min_days,tx_count,total_amount,1_amount,0_amount,total_amount_avg,1_amount_avg,0_amount_avg,total_amount_avg2,1_amount_avg2,0_amount_avg2,tx_count_avg,tx_tmstp_max,tx_tmstp_min
id,,,,,,,,,,,,,,
0,163,48,71787.000000,12079.500000,59707.500000,440.411043,74.107362,366.303681,1495.562500,251.656250,1243.906250,0.294479,1238198400,1224115200
2,180,48,22406.100000,15883.720000,6522.380000,124.478333,88.242889,36.235444,466.793750,330.910833,135.882917,0.266667,1383955200,1368403200
4,169,93,51163.000000,30823.100000,20339.900000,302.739645,182.385207,120.354438,550.139785,331.431183,218.708602,0.550296,1238284800,1223683200
6,179,61,41733.770000,15385.270000,26348.500000,233.149553,85.951229,147.198324,684.160164,252.217541,431.942623,0.340782,1220227200,1204761600
7,175,66,59958.010000,22642.760000,37315.250000,342.617200,129.387200,213.230000,908.454697,343.072121,565.382576,0.377143,1201996800,1186876800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53465,167,12,2453.143203,2453.143203,0.000000,14.689480,14.689480,0.000000,204.428600,204.428600,0.000000,0.071856,1161475200,1147046400
53467,180,281,151215.683895,88061.985230,63153.698666,840.087133,489.233251,350.853881,538.134106,313.387848,224.746259,1.561111,1161475200,1145923200
53468,180,333,418948.653631,333591.040096,85357.613535,2327.492520,1853.283556,474.208964,1258.104065,1001.774895,256.329170,1.850000,1161475200,1145923200


In [75]:
df_tx_stat.shape

(21728, 14)

# 合并特征

In [76]:
df_concat = pd.concat([df, df_tx_stat], axis=1)
print(df_concat.shape)
df_concat.reset_index(inplace=True)

(53480, 59)


In [84]:
df_concat = df_concat.rename(columns={'index':'id'})
print('isna \n',df_concat.isna().sum())

col_str = 'tx_max_min_days,tx_count,total_amount,1_amount,0_amount,total_amount_avg,1_amount_avg,0_amount_avg,total_amount_avg2,1_amount_avg2,0_amount_avg2,tx_count_avg,tx_tmstp_max,tx_tmstp_min' 
for col_name in col_str.split(',') : 
    value = df_concat[col_name].mean()
    print(col_name, value)
#     df_concat = df_concat.fillna(value)
    
print('isna \n',df_concat.isna().sum())

# df_concat.to_csv(output_path, index=False)
# print('output_path : ', output_path)
# print('process time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))
# !ls

isna 
 id                                0
title                             0
career                         2608
zip_code                          0
residence                         0
loan                              0
term                              0
interest_rate                     0
issue_time                        0
syndicated                        0
installment                       0
record_time                       0
history_time                      0
total_accounts                    0
balance_accounts                  0
balance_limit                   130
balance                           0
level                             0
label                             0
balance_account_avg               0
loan_term_avg                     0
balance_accounts_ratio            0
loan_log                          0
balance_account_avg_log           0
loan_term_avg_log                 0
balance_accounts_ratio_log        0
interest_rate_log                 0
balance_log          

In [53]:
!wc -l $output_path

20055 test.dat.202508261937


In [54]:
print('done  time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))
df_concat

done  time :  2025-08-27 00:32:05


,id,title,career,zip_code,residence,loan,term,interest_rate,issue_time,syndicated,...,tx_count,total_amount,1_amount,0_amount,total_amount_avg,1_amount_avg,0_amount_avg,total_amount_avg2,1_amount_avg2,0_amount_avg2
0,53480,10,5.0,512068,0,5950,36,14.10,1054600001,0,...,46.0,61815.52,11245.52,50570.00,374.639515,68.154667,306.484848,1343.815652,244.467826,1099.347826
1,53481,8,10.0,401852,2,15350,36,5.25,1172880047,0,...,56.0,38555.84,20161.94,18393.90,267.748889,140.013472,127.735417,688.497143,360.034643,328.462500
2,53482,8,10.0,321693,0,11050,36,13.65,1238631003,0,...,74.0,81589.80,44977.46,36612.34,474.359302,261.496860,212.862442,1102.564865,607.803514,494.761351
3,53483,8,7.0,231397,0,15350,36,14.20,1136161107,0,...,73.0,48205.32,24048.52,24156.80,303.178113,151.248553,151.929560,660.346849,329.431781,330.915068
4,53484,7,7.0,171193,0,7800,36,10.00,1214960354,0,...,91.0,78113.61,29797.86,48315.75,448.928793,171.252069,277.676724,858.391319,327.449011,530.942308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20049,73529,0,8.0,601107,1,10000,12,18.85,1130976000,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20050,73530,0,10.0,601102,1,10000,12,29.30,1156204800,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20051,73531,0,4.0,601408,1,11000,12,24.75,1144108800,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20052,73532,0,3.0,601904,1,8000,12,22.00,1163808000,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
